### Importación de librerías

Importamos las librerías necesarias para el procesamiento y análisis de datos:

- `pandas`: utilizado para la manipulación de datos en estructuras tipo DataFrame.
- `numpy`: proporciona funciones numéricas y estructuras de datos eficientes.
- `polars`: alternativa a pandas, optimizada para velocidad y procesamiento columnar.
- `itertools.chain`: para concatenar iterables de forma eficiente.
- `re`: módulo de expresiones regulares para procesamiento y limpieza de texto.

In [1]:
!pip install polars


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import polars as pl
from itertools import chain
import re

## Carga de datos

Cargamos el dataset en excel utilizando `pandas` para hacer el preprocesado y hacemos una vista general.

In [3]:
df = pd.read_excel('data.xlsx')

In [4]:
df.head(1)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0


### Listado de tareas de preprocesamiento

A continuación se resumen los pasos clave para preparar el dataset antes del análisis o modelado:

- **ID**: eliminar esta columna porque es solo un identificador único que no aporta información predictiva.

- **Patient Sex**: transformar a variable binaria para que el modelo pueda interpretarla fácilmente.

- **Left-Fundus** y **Right-Fundus**: mantener estas columnas tal cual, pues contienen la ruta a las imágenes.

- **Left-Diagnostic Keywords** y **Right-Diagnostic Keywords**: se evaluará vectorizar estas columnas usando técnicas como TOP-N para representar palabras clave.

- **N, D, G, C, A, H, M, O**: estas columnas corresponden a las clases objetivo (target) para la predicción.

 

####  Eliminamos la columna `ID`


In [5]:
df = df.drop(columns=['ID'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Patient Sex                3500 non-null   object
 2   Left-Fundus                3500 non-null   object
 3   Right-Fundus               3500 non-null   object
 4   Left-Diagnostic Keywords   3500 non-null   object
 5   Right-Diagnostic Keywords  3500 non-null   object
 6   N                          3500 non-null   int64 
 7   D                          3500 non-null   int64 
 8   G                          3500 non-null   int64 
 9   C                          3500 non-null   int64 
 10  A                          3500 non-null   int64 
 11  H                          3500 non-null   int64 
 12  M                          3500 non-null   int64 
 13  O                          3500 non-null   int64 
dtypes: int64

#### Transformamos `Patient_Sex_Binario` a binario

In [7]:
#Transformar a binario el campo Patient Sex
df['Patient_Sex_Binario'] = df['Patient Sex'].map({'Male': 1, "Female": 0})


In [8]:
df.Patient_Sex_Binario.head(10)

0    0
1    1
2    1
3    1
4    1
5    0
6    1
7    0
8    1
9    1
Name: Patient_Sex_Binario, dtype: int64

In [9]:
df.drop(columns=['Patient Sex'], inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Left-Fundus                3500 non-null   object
 2   Right-Fundus               3500 non-null   object
 3   Left-Diagnostic Keywords   3500 non-null   object
 4   Right-Diagnostic Keywords  3500 non-null   object
 5   N                          3500 non-null   int64 
 6   D                          3500 non-null   int64 
 7   G                          3500 non-null   int64 
 8   C                          3500 non-null   int64 
 9   A                          3500 non-null   int64 
 10  H                          3500 non-null   int64 
 11  M                          3500 non-null   int64 
 12  O                          3500 non-null   int64 
 13  Patient_Sex_Binario        3500 non-null   int64 
dtypes: int64

- opcional crear columna categorica con la cantidad de enfermedades.
- opcional crear columna Binaria con si es lens dust o no 0/1

####  Procesamos palabras clave (Keywords)

Definimos un patrón de separación para dividir las palabras clave en las columnas:

- Separadores considerados:  
  - Coma ASCII `,`  
  - Coma china `，` (Unicode `\uFF0C`)  
  - Punto y coma `;`  
  - Barra vertical `|`

Estraemos las listas de keywords limpias en dos nuevas columnas:

- `'kw_left'` para la columna `'Left-Diagnostic Keywords'`.
- `'kw_right'` para la columna `'Right-Diagnostic Keywords'`.

In [11]:
# patrón: coma ASCII ,  |  coma china ，  |  punto y coma ;  |  barra vertical |
SEP_RE = r'\s*[,\uFF0C;|]+\s*'

def parse_keywords(cell: str) -> list[str]:
   
    if pd.isna(cell) or not cell.strip():
        return []
    parts = re.split(SEP_RE, cell.strip().lower())
    return [p for p in parts if p]

df['kw_left']  = df['Left-Diagnostic Keywords'].apply(parse_keywords)
df['kw_right'] = df['Right-Diagnostic Keywords'].apply(parse_keywords)

In [12]:
all_kw = sorted(set(chain.from_iterable(df['kw_left'])) | 
                set(chain.from_iterable(df['kw_right'])))

PAD_IDX = 0           # reservado para padding
stoi = {kw: i+1 for i, kw in enumerate(all_kw)}   # arranca en 1
itos = {i: w for w, i in stoi.items()}


#### Conversión de listas de keywords a índices numéricos

Utilizando el vocabulario `stoi`, transformamos las listas de keywords en las columnas `kw_left` y `kw_right` a listas de índices numéricos.


In [13]:
df['left_idx']  = df['kw_left'].apply(lambda kws: [stoi[k] for k in kws])
df['right_idx'] = df['kw_right'].apply(lambda kws: [stoi[k] for k in kws])


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Patient Age                3500 non-null   int64 
 1   Left-Fundus                3500 non-null   object
 2   Right-Fundus               3500 non-null   object
 3   Left-Diagnostic Keywords   3500 non-null   object
 4   Right-Diagnostic Keywords  3500 non-null   object
 5   N                          3500 non-null   int64 
 6   D                          3500 non-null   int64 
 7   G                          3500 non-null   int64 
 8   C                          3500 non-null   int64 
 9   A                          3500 non-null   int64 
 10  H                          3500 non-null   int64 
 11  M                          3500 non-null   int64 
 12  O                          3500 non-null   int64 
 13  Patient_Sex_Binario        3500 non-null   int64 
 14  kw_left 

In [15]:
df.head(5)           


,Patient Age,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,Patient_Sex_Binario,kw_left,kw_right,left_idx,right_idx
0,69,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,0,[cataract],[normal fundus],[10],[50]
1,57,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,1,[normal fundus],[normal fundus],[50],[50]
2,42,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,1,"[laser spot, moderate non proliferative retino...",[moderate non proliferative retinopathy],"[34, 43]",[43]
3,66,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,1,[normal fundus],[branch retinal artery occlusion],[50],[8]
4,53,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,1,[macular epiretinal membrane],[mild nonproliferative retinopathy],[38],[42]


Eliminamos las columnas que no son necesarias para nuestro dataset final:

- `'Left-Diagnostic Keywords'`
- `'Right-Diagnostic Keywords'`
- `'kw_left'`
- `'kw_right'`

In [16]:
df.drop(columns=['Left-Diagnostic Keywords', 'Right-Diagnostic Keywords','kw_left','kw_right'], inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Patient Age          3500 non-null   int64 
 1   Left-Fundus          3500 non-null   object
 2   Right-Fundus         3500 non-null   object
 3   N                    3500 non-null   int64 
 4   D                    3500 non-null   int64 
 5   G                    3500 non-null   int64 
 6   C                    3500 non-null   int64 
 7   A                    3500 non-null   int64 
 8   H                    3500 non-null   int64 
 9   M                    3500 non-null   int64 
 10  O                    3500 non-null   int64 
 11  Patient_Sex_Binario  3500 non-null   int64 
 12  left_idx             3500 non-null   object
 13  right_idx            3500 non-null   object
dtypes: int64(10), object(4)
memory usage: 382.9+ KB


In [20]:
#nos quedamos con los pacientes mayores de 1 año
df = df[df['Patient Age'] > 1]


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3484 entries, 0 to 3499
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Patient Age          3484 non-null   int64 
 1   Left-Fundus          3484 non-null   object
 2   Right-Fundus         3484 non-null   object
 3   N                    3484 non-null   int64 
 4   D                    3484 non-null   int64 
 5   G                    3484 non-null   int64 
 6   C                    3484 non-null   int64 
 7   A                    3484 non-null   int64 
 8   H                    3484 non-null   int64 
 9   M                    3484 non-null   int64 
 10  O                    3484 non-null   int64 
 11  Patient_Sex_Binario  3484 non-null   int64 
 12  left_idx             3484 non-null   object
 13  right_idx            3484 non-null   object
dtypes: int64(10), object(4)
memory usage: 408.3+ KB


Tenemos las columnas `left_idx` y `right_idx`, que contienen datos en forma de listas.

En este caso, **no realizaremos ninguna transformación adicional**, ya que PyTorch permite convertir listas directamente a tensores utilizando `torch.tensor()`:

### Guardamos el dataset en formato Parquet

Se eligió Parquet por ser un formato óptimo para análisis de datos a escala.

- **Eficiente**: ocupa menos espacio que CSV gracias a la compresión columnar.
- **Rápido**: permite lectura y escritura más rápida, especialmente al trabajar con columnas específicas.
- **Compatible**: ampliamente soportado por pandas, PySpark (que usaremos mas adelante) y otras herramientas de análisis.


In [22]:
df_pl = pl.from_pandas(df, rechunk=True)

In [24]:
df_pl.write_parquet(
    "dataset_meta_final.parquet",
    compression="zstd",        # 'snappy' por defecto; zstd comprime ~25-40 % más
    compression_level=5        # 1-22; 5-7 suele ser buen balance
)